# Capstone Project - Week 1
## Function 5: 4D Optimization Challenge
### Initial Data Exploration and Analysis

**Date:** January 27, 2026

**Objective:** Maximize F5 using Bayesian Optimization

**Approach:** 
- Load and explore initial 4D samples
- Fit Gaussian Process surrogate model
- Use Expected Improvement for next point selection
- Prepare strategy for Week 2

## Section 1: Import Required Libraries

In [ ]:
# Core scientific computing libraries
import numpy as np
import matplotlib.pyplot as plt

# Gaussian Process regression and optimization
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel, Matern
from scipy.stats import norm
from scipy.optimize import minimize

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print(f'Libraries loaded successfully for F5 (4D)')

## Section 2: Load Initial Data

Loading Week 1 initial samples for Function 5

In [ ]:
# Define file paths for F5
INPUT_FILE = 'f5_initial_inputs.npy'   # Shape: (n_samples, 4)
OUTPUT_FILE = 'f5_initial_outputs.npy' # Shape: (n_samples,)

In [ ]:
# Load data
X = np.load(INPUT_FILE)  # Input matrix
Y = np.load(OUTPUT_FILE) # Output vector

# Display data information
print('F5 Data Loaded:')
print(f'  Input shape:  {X.shape}')
print(f'  Output shape: {Y.shape}')
print(f'  Dimensions:   {X.shape[1]}D')
print(f'  Samples:      {len(X)}')

## Section 3: Exploratory Data Analysis

In [ ]:
# Ensure proper data format
X = np.atleast_2d(X)
Y = Y.ravel()

print('='*70)
print('FUNCTION 5 - INITIAL DATA ANALYSIS')
print('='*70)

# Output statistics
print(f'\nOutput Statistics:')
print(f'  Maximum:  {Y.max():.6f}')
print(f'  Minimum:  {Y.min():.6f}')
print(f'  Mean:     {Y.mean():.6f}')
print(f'  Median:   {np.median(Y):.6f}')
print(f'  Std Dev:  {Y.std():.6f}')
print(f'  Range:    {Y.max() - Y.min():.6f}')

# Find best point
best_idx = np.argmax(Y)
best_value = Y[best_idx]
best_point = X[best_idx]

print(f'\n🎯 Best Point Found (Initial Sampling):')
print(f'  Value:    {best_value:.6f}')
print(f'  Location: {best_point}')

# Input space coverage
print(f'\nInput Space Coverage:')
for i in range(X.shape[1]):
    print(f'  X{i+1}: [{X[:, i].min():.4f}, {X[:, i].max():.4f}]')

In [ ]:
##now print all the data
print("Input function 5 data: \n",np.load(INPUT_FILE))
print("Output function 5 data: \n",np.load(OUTPUT_FILE))

In [ ]:
print(f"Loaded {X.shape[0]} points in {X.shape[1]}D" )
print(f"Current best : {Y.max():.6e} at {X[Y.argmax()]}")  # 
X = np.atleast_2d(X)  # Make sure X is aleays 4d
Y = Y.ravel() #  forces Y to be 1 d flat


In [ ]:
##Section 2: Fit surrogate for function 1

## Section 4: Gaussian Process Model

Fitting a GP surrogate model to learn the function landscape

In [ ]:
kernel  = ConstantKernel(1.0,constant_value_bounds=(1e-10,1e10)) * RBF(length_scale=0.3,  length_scale_bounds=(0.01,10.0)) \
+ WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-9,10000.0))

In [ ]:
gp = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer = 30, normalize_y=False, random_state=42)

In [ ]:
gp.fit(X,Y)

In [ ]:
##Section 3  Define the acquision function

## Section 6: Visualization

Visualizing the optimization landscape and strategy

In [ ]:
# create grid for plotting

In [ ]:
n_points = 200          # lower than 1000 because 4D → memory & time
x1 = np.linspace(X[:,0].min(), X[:,0].max(), n_points)

x_plot = np.column_stack([
    x1,
    np.full(n_points, X[:,1].mean()),
    np.full(n_points, X[:,2].mean()),
     np.full(n_points, X[:,3].mean())
])

In [ ]:
# Get GP Predictions 

In [ ]:
mv, sigma = gp.predict(x_plot, return_std=True)

## Section 5: Acquisition Function Optimization

Using Expected Improvement to select next sampling point

In [ ]:
# Calculate expected improvement

In [ ]:
y_best = Y.max() # best observed value so far

In [ ]:
# EI CALCULATION

In [ ]:
with np.errstate(divide='warn'):
    improvement = mv - y_best
    Z = improvement / sigma
    EI = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    EI[sigma == 0.0] = 0.0

In [ ]:
# FIND NEXT SAMPLING POINT 

In [ ]:
x_next = x_plot[np.argmax(EI)]

In [ ]:
print(f"Best current value :{y_best:.6f} at x{X[Y.argmax()][0]:.6f}")

In [ ]:
print(f"EI data gives next point to sample: x_next = {x_next[0]:.6f},{x_next[1]:.6f}")

In [ ]:
#Now Plot

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10)) 
# Top plot: GP surrogate 
ax1.plot(x_plot[:, 0], mv, 'b-', label='GP Mean', linewidth=2) 
ax1.fill_between(x_plot[:, 0], mv - 1.96*sigma, mv + 1.96*sigma, alpha=0.2, color='blue', label='95% Confidence') 
ax1.scatter(X[:, 0], Y, c='red', s=80, zorder=10, label='Observations') 
ax1.axhline(y=y_best, color='green', linestyle='--', linewidth=2, label=f'Best: {y_best:.4f}') 
ax1.set_xlabel('x₁') 
ax1.set_ylabel('f(x)') 
ax1.set_title('Gaussian Process Surrogate Model') 
ax1.legend() 
ax1.grid(True, alpha=0.3) 
# Bottom plot: Acquisition function (EI)
ax2.plot(x_plot[:, 0], EI, 'purple', linewidth=2.5, label='Expected Improvement') 
ax2.axvline(x=x_next[0], color='red', linestyle='--', linewidth=2) 
ax2.scatter(x_next[0], np.max(EI), c='red', s=200, marker='*', zorder=10, label=f'Next point') 
ax2.fill_between(x_plot[:, 0], 0, EI, alpha=0.3, color='purple') 
ax2.set_xlabel('x₁') 
ax2.set_ylabel('Acquisition Value') 
ax2.set_title('Acquisition Function (Expected Improvement)') 
ax2.legend() 
ax2.grid(True, alpha=0.3) 
plt.tight_layout() 
plt.show()

In [ ]:
##Now look for the UCB

In [ ]:
# UCB Acquisition Function - Create 4D prediction grid
kappa = 2.0  # Exploration parameter

# Create grid for UCB (X1 varies, others fixed at mean)
n_points = 300
x1_ucb = np.linspace(X[:, 0].min(), X[:, 0].max(), n_points)

x_plot_ucb = np.column_stack([
    x1_ucb,
    np.full(n_points, X[:, 1].mean()),
    np.full(n_points, X[:, 2].mean()),
    np.full(n_points, X[:, 3].mean()),
])

print(f'UCB grid created: {x_plot_ucb.shape}')
print(f'Grid is 4D: X1 varies, others fixed at their means')


In [ ]:
##Find Next sampling point

In [ ]:
# Generate predictions on UCB grid
mv_ucb, sigma_ucb = gp.predict(x_plot_ucb, return_std=True)
UCB = mv_ucb + kappa * sigma_ucb

# Find best UCB point
x_next_idx = np.argmax(UCB)
x_next_ucb = x_plot_ucb[x_next_idx]
y_best_ucb = Y.max()

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Top plot: GP surrogate
ax1.plot(x_plot_ucb[:, 0], mv_ucb, 'b-', label='GP Mean', linewidth=2)
ax1.fill_between(x_plot_ucb[:, 0], mv_ucb - 1.96*sigma_ucb, mv_ucb + 1.96*sigma_ucb,
                 alpha=0.2, color='blue', label='95% Confidence')
ax1.scatter(X[:, 0], Y, c='red', s=80, zorder=10, label='Observations')
ax1.axhline(y=y_best_ucb, color='green', linestyle='--', linewidth=2,
            label=f'Best: {y_best_ucb:.4f}')
ax1.set_xlabel('X₁')
ax1.set_ylabel('f(x)')
ax1.set_title('F5 - Gaussian Process Surrogate Model (4D)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Bottom plot: UCB acquisition function
ax2.plot(x_plot_ucb[:, 0], UCB, 'purple', linewidth=2.5,
         label=f'UCB (κ={kappa})')
ax2.plot(x_plot_ucb[:, 0], mv_ucb, 'b--', linewidth=1.5, alpha=0.6,
         label='GP Mean')
ax2.axvline(x=x_next_ucb[0], color='red', linestyle='--', linewidth=2)
ax2.scatter(x_next_ucb[0], np.max(UCB), c='red', s=200, marker='*',
            zorder=10, edgecolors='black', linewidth=2.5,
            label=f'Next point: {x_next_ucb[0]:.3f}')
ax2.fill_between(x_plot_ucb[:, 0], 0, UCB, alpha=0.3, color='purple')
ax2.set_xlabel('X₁', fontsize=12)
ax2.set_ylabel('Acquisition Value', fontsize=12)
ax2.set_title('F5 - Upper Confidence Bound Acquisition', fontsize=14, fontweight='bold')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\nUCB suggests next point for F5: {x_next_ucb}')


In [ ]:
# Format the next query point according to assignment requirements 
# Each xi must begin with 0 and have 6 decimal places 
# Extract the coordinates 
x1_next_ucb = x_next_ucb[0] 
x2_next_ucb = x_next_ucb[1] 

In [ ]:
# Format to 6 decimal places, ensuring they start with 0 
query_string_ucb = f"0.{str(x1_next_ucb).split('.')[1][:6]}-0.{str(x2_next_ucb).split('.')[1][:6]}"


In [ ]:
# Now print the next best sampling points for UCB and EI

In [ ]:
print("\n" + "="*60) 
print("QUERY SUBMISSION") 
print("="*60) 
print(f"UCB data gives next point to sample for Function 1: x_next_ucb = {x_next_ucb[0]:.6f}, {x_next_ucb[1]:.6f},{x_next_ucb[2]:.6f},{x_next_ucb[3]:.6f}") 
print(f"EI  data gives next point to sample for Function 1: x_next = {x_next[0]:.6f}, {x_next[1]:.6f},{x_next[2]:.6f},{x_next[3]:.6f}")
print("="*60)

In [ ]:
 # generate random data points
from scipy.stats import norm

def expected_improvement(X_new,gp,y_best,xi=0.01):
     mv,sigma= gp.predict(X_new,return_std=True)
     sigma = np.maximum(sigma,1e-10)
     imp = mv-y_best -xi
     Z = imp/sigma
     ei = imp*norm.cdf(Z)+sigma * norm.pdf(Z)
     ei[sigma < 1e-10] =0.0
     return ei

kernel  = ConstantKernel(1.0,constant_value_bounds=(1e-1,1e10)) * RBF(length_scale=0.3,  length_scale_bounds=(0.05,10.0)) \
+ WhiteKernel(noise_level=1e-15, noise_level_bounds=(1e-15,1e-2))
gp = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer = 30, normalize_y=False, random_state=42)
gp.fit(X,Y)
n_candidates = 5000
candidates = np.random.uniform(0,1,size=(n_candidates,4))

#compute acquision curve on candidates

acquisition_type = "UCB"
#acquisition_type = "EI"

best_idx= None

if acquisition_type == "EI":
    y_best = Y.max()
    ei_values = expected_improvement(candidates,gp,y_best=y_best,xi=0.01)
    best_idx=np.argmax(ei_values)
    print("Using Expected Improvement (EI)")
elif  acquisition_type == "UCB":
      mv,sigma=gp.predict(candidates,return_std=True)
      ucb_values = mv+2.0*sigma
      best_idx = np.argmax(ucb_values)
      print("Using Upper Confidence Bounds (UCB)")
else:
    raise ValueError("Invalid acquision type")
if acquisition_type == "UCB":
    x_next = candidates[best_idx]
    x_next = np.clip(x_next,0.0,1.0)
    
print("Random data: Next point from random candidates:",x_next)
print("Random Data: GP fitted. Learned kernal :",gp.kernel_)

## Section 7: Week 1 Summary and Next Steps

### Key Findings:
- Function 5 operates in 4D space
- Initial exploration completed with Gaussian Process model
- Expected Improvement acquisition function identified next sampling point

### Week 2 Strategy:
- Test the recommended point from EI optimization
- Update GP model with new data
- Refine optimization strategy based on results
- Consider local vs global exploration trade-offs

### F5 Specific Observations:
- Initial sample size allows for basic GP fitting
- 4D search space requires intelligent sampling
- Bayesian Optimization essential for efficient exploration